# Introvert or Extrovert

## Importing Libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Read the Dataset

In [4]:
train_df = pd.read_csv("../datasets/train.csv")
test_df = pd.read_csv("../datasets/test.csv")

In [ ]:
print(train_df.shape)
print(train_df.isnull().sum())
train_df.head()

(18524, 9)


id                              0
Time_spent_Alone             1190
Stage_fear                   1893
Social_event_attendance      1180
Going_outside                1466
Drained_after_socializing    1149
Friends_circle_size          1054
Post_frequency               1264
Personality                     0
dtype: int64

In [20]:
print(test_df.shape)
print(test_df.isnull().sum())
test_df.head()

(6175, 8)
id                             0
Time_spent_Alone             425
Stage_fear                   598
Social_event_attendance      397
Going_outside                466
Drained_after_socializing    432
Friends_circle_size          350
Post_frequency               408
dtype: int64


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,No,7.0,4.0,No,6.0,NaN
1,18525,NaN,Yes,0.0,0.0,Yes,5.0,1.0
2,18526,3.0,No,5.0,6.0,No,15.0,9.0
3,18527,3.0,No,4.0,4.0,No,5.0,6.0
4,18528,9.0,Yes,1.0,2.0,Yes,1.0,1.0


## Merge Train and Test Data

In [12]:
X = train_df.drop(columns = 'Personality', axis = 1)

df_merged = pd.merge(X, test_df, how = 'outer')

In [21]:
df_merged.isnull().sum()

id                              0
Time_spent_Alone             1615
Stage_fear                   2491
Social_event_attendance      1577
Going_outside                1932
Drained_after_socializing    1581
Friends_circle_size          1404
Post_frequency               1672
dtype: int64

## Encoding Values

In [ ]:
df_merged = df_merged.replace({'Yes': 1, 'No': 0,})

/tmp/ipykernel_55002/2597517951.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged = df_merged.replace({'Yes': 1, 'No': 0,})


## Imlputing Values

In [24]:
df_merged.columns

Index(['id', 'Time_spent_Alone', 'Stage_fear', 'Social_event_attendance',
       'Going_outside', 'Drained_after_socializing', 'Friends_circle_size',
       'Post_frequency'],
      dtype='object')

In [25]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy = 'mean')

num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside', 'Friends_circle_size', 'Post_frequency']

df_merged[num_cols] = si.fit_transform(df_merged[num_cols])

cat_cols = ['Stage_fear', 'Drained_after_socializing']

cat_imputer = SimpleImputer(strategy = 'most_frequent')

df_merged[cat_cols] = cat_imputer.fit_transform(df_merged[cat_cols])
